In [90]:
#access the csv called batting_season_summary.csv in example_data:

import pandas as pd
import numpy as np

df = pd.read_csv('example_data/batting_season_summary.csv')




In [91]:
#alrigth so we have the columns of years
# i want to rearrange it, such that each row has the player with a data for year x, and then the data for year x+1
#for example, if there used to be two rows like:
# player1, data1, data2, data3, 2014
# player1, data4, data5, data6, 2015
#player1, data8, data9, data10, 2016
#then this should make 2 rows like:
#player1, data1, data2, data3, 2014, data4, data5, data6, 2015
#player1, data4, data5, data6, 2015, data8, data9, data10, 2016
#anyways, for a particular player who has entries for n years, there should be n-1 rows for that player in the new dataframe.
#heres the code:

#no more comments, heres the code:

#sort the df by player and year
df = df.sort_values(by=['Name', 'Year'])
#drop the columns 'team' and 'pos' 
df = df.drop(columns=['team', 'pos'])


In [92]:
print(df.shape)

(2700, 16)


In [62]:
#alright do a neural network taking all this data (except name and year) and predict H. use pytorch:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

#for reference theres 14 columns in the df, 1 of them is the target (H). everything has been dropped so just use. heres the code:

#split the data into train and test
X = df.drop(columns=['H', 'BA'] ).values
y = df['H'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#convert the data to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

#make a dataloader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

#make the neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(X.shape[1], 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model = Net()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#train the model
for epoch in range(100):
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch.view(-1, 1))
        loss.backward()
        optimizer.step()
    print(f'epoch {epoch+1}, loss: {loss.item()}')

#evaluate the model
model.eval()
with torch.no_grad():
    y_pred = model(X_test)
    loss = criterion(y_pred, y_test.view(-1, 1))
    print(f'test loss: {loss.item()}')
    print(f'test mse: {mean_squared_error(y_test, y_pred)}')

#save the model
torch.save(model.state_dict(), 'model.pth')


epoch 1, loss: 26542.6875
epoch 2, loss: 20002.330078125
epoch 3, loss: 17104.216796875
epoch 4, loss: 7737.54833984375
epoch 5, loss: 3047.875732421875
epoch 6, loss: 2668.345947265625
epoch 7, loss: 2135.83154296875
epoch 8, loss: 1949.6180419921875
epoch 9, loss: 1670.4635009765625
epoch 10, loss: 999.3551635742188
epoch 11, loss: 1261.3460693359375
epoch 12, loss: 1081.0579833984375
epoch 13, loss: 683.124755859375
epoch 14, loss: 757.15234375
epoch 15, loss: 798.0426635742188
epoch 16, loss: 473.7430419921875
epoch 17, loss: 562.0117797851562
epoch 18, loss: 484.504638671875
epoch 19, loss: 418.5822448730469
epoch 20, loss: 352.607666015625
epoch 21, loss: 307.84930419921875
epoch 22, loss: 534.7980346679688
epoch 23, loss: 288.90997314453125
epoch 24, loss: 227.56394958496094
epoch 25, loss: 184.62200927734375
epoch 26, loss: 192.52256774902344
epoch 27, loss: 102.4846420288086
epoch 28, loss: 117.33736419677734
epoch 29, loss: 113.6229248046875
epoch 30, loss: 95.54291534423828


0.9987925900738972


{'max_depth': 8, 'n_estimators': 70}
0.9669703256409132
36.785861681433786
0.9594750342526668


In [174]:
import pandas as pd
from math import radians, sin, cos, sqrt, atan2

# Function to calculate distance between two geographical coordinates using Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    # Radius of the Earth in km
    R = 6371.0
    
    # Convert latitude and longitude from degrees to radians
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    
    # Calculate the change in coordinates
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    # Apply Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    
    return distance

# Function to get average distance traveled for a given year and team
def get_average_distance(year, team):
    # Load schedule data for the given year
    schedule_filename = f'example_data/schedules/{year}_schedule.csv'
    total_distance = 0
    games_played = 0
    
    # Parse the schedule and calculate distances
    with open(schedule_filename, 'r') as file:
        next(file)  # Skip header
        for line in file:
            schedule_year, _, home_team, away_team = line.strip().split(',')[0:4]
            
            # Check if it's an away game for the given team
            if schedule_year == year and away_team == team:
                # Check if coordinates for both cities are available
                if home_team in city_coordinates and away_team in city_coordinates:
                    home_lat, home_lon = city_coordinates[home_team]
                    away_lat, away_lon = city_coordinates[away_team]
                    
                    # Calculate distance between home and away cities
                    distance = calculate_distance(home_lat, home_lon, away_lat, away_lon)
                    total_distance += distance
                    games_played += 1
    
    # Calculate average distance traveled
    if games_played > 0:
        return total_distance / games_played
    else:
        return None

# Dictionary to store the coordinates of each city
city_coordinates = {
    'Royals': (39.0513, -94.4805),     # Kansas City (Royals)
    'Braves': (33.8908, -84.4679),     # Atlanta (Braves)
    'Rays': (27.7684, -82.6483),       # St. Petersburg (Rays)
    'Blue Jays': (43.6414, -79.3894),  # Toronto (Blue Jays)
    'Diamondbacks': (33.4455, -112.0667),  # Phoenix (Diamondbacks)
    'Astros': (29.7572, -95.3554),     # Houston (Astros)
    'Pirates': (40.4469, -80.0057),    # Pittsburgh (Pirates)
    'Dodgers': (34.0736, -118.2400),   # Los Angeles (Dodgers)
    'Rockies': (39.7554, -104.9881),   # Denver (Rockies)
    'Nationals': (38.8729, -77.0074),  # Washington D.C. (Nationals)
    'Cardinals': (38.6226, -90.1928),  # St. Louis (Cardinals)
    'Red Sox': (42.3467, -71.0972),    # Boston (Red Sox)
    'Orioles': (39.2839, -76.6217),    # Baltimore (Orioles)
    'Giants': (37.7786, -122.3893),    # San Francisco (Giants)
    'Reds': (39.0979, -84.5086),       # Cincinnati (Reds)
    'Indians': (41.4959, -81.6853),    # Cleveland (Indians)
    'Padres': (32.7076, -117.1570),    # San Diego (Padres)
    'Phillies': (39.9054, -75.1669),   # Philadelphia (Phillies)
    'White Sox': (41.8301, -87.6347),  # Chicago (White Sox)
    'Brewers': (43.0280, -87.9712),    # Milwaukee (Brewers)
    'Yankees': (40.8296, -73.9262),    # New York City (Yankees)
    'Mets': (40.7571, -73.8458),       # New York City (Mets)
    'Rangers': (32.7511, -97.0820),    # Arlington (Rangers)
    'Marlins': (25.7780, -80.2195),    # Miami (Marlins)
    'Mariners': (47.5914, -122.3325),  # Seattle (Mariners)
    'Twins': (44.9817, -93.2784),      # Minneapolis (Twins)
    'Angels': (33.8003, -117.8827),    # Anaheim (Angels)
    'Cubs': (41.9484, -87.6553),       # Chicago (Cubs)
    'Athletics': (37.7516, -122.2005), # Oakland (Athletics)
    'Tigers': (42.3391, -83.0487)      # Detroit (Tigers
}

In [175]:
#ok this is what we're going to do. look ath the file batting_season_summary.
# Name,age,team,pos,PA,AB,H,2B,3B,HR,BB,SO,P/PA,BA,OBP,SLG,OPS,Year.

#train a neural networking taking age, team, pos, PA, AB, 2B, 3B, HR, BB, SO, P/PA, BA, OBP, SLG, OPS, SPECIFICALLY FOR YEAR 2022 as input,
#and outputs age, team, pos, PA, AB, H, 2B, 3B, HR, BB, SO, P/PA, BA, OBP, SLG, OPS for year 2023.
#basically each player who has had an entry in both 2022 and 2023 will be sort of used as data, where the 2022 one is their input and the 2023 one is their output to compare against.
#so let's make the model:

#first we need to get the data for 2022 and 2023
df = pd.read_csv('example_data/batting_season_summary.csv')
df['average_distance_traveled'] = df.apply(lambda row: get_average_distance(str(row['Year']), row['team']), axis=1)

#drop the rows that don't have 2022 or 2023
df = df[(df['Year'] == 2022) | (df['Year'] == 2023)]
#sort by name:
df = df.sort_values(by='Name')

#drop names that only have 1 entry
df = df.groupby('Name').filter(lambda x: len(x) == 2)

#now basically combine the 2022 and 2023 data for each player into one row:
df = df.pivot(index='Name', columns='Year')
df.columns = ['_'.join(map(str, col)).strip() for col in df.columns.values]
df = df.reset_index()
print(df) 



                  Name  age_2022  age_2023     team_2022     team_2023  \
0         Aaron Looper        37        38  Diamondbacks  Diamondbacks   
1       Aaron Scheffer        38        39        Astros        Astros   
2        Adam Peterson        29        30        Braves        Braves   
3            Adam Wilk        20        21       Rangers       Rangers   
4        Adrian Houser        41        42        Royals        Royals   
..                 ...       ...       ...           ...           ...   
232    Tyler Ladendorf        29        30       Rangers       Rangers   
233      Victor Garate        28        29      Phillies      Phillies   
234      Vince Belnome        30        31        Giants        Giants   
235     Wilfredo Tovar        26        27      Mariners      Mariners   
236  Wilking Rodriguez        33        34         Twins         Twins   

    pos_2022 pos_2023  PA_2022  PA_2023  AB_2022  ...   BA_2022   BA_2023  \
0          C        C      723    

In [198]:
#ok now we train a neural network to predict the 2023 data from the 2022 data:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

#the input is the 2022 data, the output is the 2023 data.
#aka the input is age_2022, team_2022, pos_2022, PA_2022, AB_2022, H_2022, 2B_2022, 3B_2022, HR_2022, BB_2022, SO_2022, P/PA_2022, BA_2022, OBP_2022, SLG_2022, OPS_2022
#the output is age_2023, team_2023, pos_2023, PA_2023, AB_2023, H_2023, 2B_2023, 3B_2023, HR_2023, BB_2023, SO_2023, P/PA_2023, BA_2023, OBP_2023, SLG_2023, OPS_2023:
#so the input has 15 columns, the output has 15 columns:

#for x, only keep the rows age_2022, team_2022, pos_2022, PA_2022, AB_2022, H_2022, 2B_2022, 3B_2022, HR_2022, BB_2022, SO_2022, P/PA_2022, BA_2022, OBP_2022, SLG_2022, OPS_2022
#and for y, only keep the rows age_2023, team_2023, pos_2023, PA_2023, AB_2023, H_2023, 2B_2023, 3B_2023, HR_2023, BB_2023, SO_2023, P/PA_2023, BA_2023, OBP_2023, SLG_2023, OPS_2023
#drop team from df
#drop the team_2022 and team_2023 columns:


#split the data into X and y
X = df[['age_2022', 'PA_2022', 'AB_2022', 'H_2022', '2B_2022', '3B_2022', 'HR_2022', 'BB_2022', 'SO_2022', 'P/PA_2022', 'BA_2022', 'OBP_2022', 'SLG_2022', 'OPS_2022', 'average_distance_traveled_2022']].values
y = df[['H_2023']].values

# in x we have pos_2022, we need to convert this to a one hot encoding
# from sklearn.preprocessing import OneHotEncoder
# onehotencoder = OneHotEncoder()
# pos_2022 = onehotencoder.fit_transform(df[['pos_2022']]).toarray()
# X = np.concatenate((X, pos_2022), axis=1)

# #drop the pos_2022 column
# X = np.delete(X, 1, axis=1)

# #print column names of X



#split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = scaler.fit_transform(y_train)
y_test = scaler.transform(y_test)

#convert the data to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

#make a dataloader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

#make the neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(X.shape[1], 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, y.shape[1])

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model = Net()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.000125)

#train the model
for epoch in range(100):
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
    print(f'epoch {epoch+1}, loss: {loss.item()}')

#evaluate the model
model.eval()

with torch.no_grad():
    y_pred = model(X_test)
    loss = criterion(y_pred, y_test)
    print(f'test loss: {loss.item()}')
    print(f'test mse: {mean_squared_error(y_test, y_pred)}')

#save the model
torch.save(model.state_dict(), 'model.pth')

#print r2 of the model
from sklearn.metrics import r2_score
print(r2_score(y_test, y_pred))




epoch 1, loss: 1.038475513458252
epoch 2, loss: 0.7906256318092346
epoch 3, loss: 0.8985239863395691
epoch 4, loss: 1.024081826210022
epoch 5, loss: 1.0124821662902832
epoch 6, loss: 0.8748965263366699
epoch 7, loss: 0.7265164852142334
epoch 8, loss: 0.9148544073104858
epoch 9, loss: 0.9607894420623779
epoch 10, loss: 0.7959932088851929
epoch 11, loss: 1.0125590562820435
epoch 12, loss: 0.7897387742996216
epoch 13, loss: 0.7406659722328186
epoch 14, loss: 0.6278113722801208
epoch 15, loss: 0.5741518139839172
epoch 16, loss: 0.5436609387397766
epoch 17, loss: 0.6064728498458862
epoch 18, loss: 0.590688943862915
epoch 19, loss: 0.629690945148468
epoch 20, loss: 0.5396530628204346
epoch 21, loss: 0.7049775123596191
epoch 22, loss: 0.6235377788543701
epoch 23, loss: 0.6421233415603638
epoch 24, loss: 0.6262005567550659
epoch 25, loss: 0.5907728672027588
epoch 26, loss: 0.6454694867134094
epoch 27, loss: 0.5638565421104431
epoch 28, loss: 0.5927125215530396
epoch 29, loss: 0.565711975097656

In [178]:
#use the model to print the top 3 players who are predicted to have the most hits in 2023:
#heres the code:

#load the model
model = Net()
model.load_state_dict(torch.load('model.pth'))

#iterate through the df:


#set new_input to first row, with these columns: ['age_2022',  'PA_2022', 'AB_2022', 'H_2022', '2B_2022', '3B_2022', 'HR_2022', 'BB_2022', 'SO_2022', 'P/PA_2022', 'BA_2022', 'OBP_2022', 'SLG_2022', 'OPS_2022', 'average_distance_traveled_2022']
#set new_input to a tensor
best_player_name = None
max_hits = 0

real_best_player_name = None
real_max_hits = 0
for i in range(len(df)):
    new_input = torch.tensor(df[['age_2022', 'PA_2022', 'AB_2022', 'H_2022', '2B_2022', '3B_2022', 'HR_2022', 'BB_2022', 'SO_2022', 'P/PA_2022', 'BA_2022', 'OBP_2022', 'SLG_2022', 'OPS_2022', 'average_distance_traveled_2022']].values[i], dtype=torch.float32)

    with torch.no_grad():
        output = model(new_input)
        print(output, df['H_2023'].values[i])
        if output > max_hits:
            max_hits = output
            best_player_name = df['Name'].values[i]
        if df['H_2023'].values[i] > real_max_hits:
            real_max_hits = df['H_2023'].values[i]
            real_best_player_name = df['Name'].values[i]
print(best_player_name, max_hits)
print(real_best_player_name, real_max_hits)
        




tensor([286.9218]) 157
tensor([266.0181]) 168
tensor([245.0566]) 162
tensor([234.7612]) 160
tensor([203.9346]) 144
tensor([315.8021]) 147
tensor([194.4701]) 122
tensor([229.9673]) 100
tensor([296.9379]) 139
tensor([237.2370]) 101
tensor([281.7636]) 97
tensor([264.7977]) 129
tensor([234.4664]) 96
tensor([247.1158]) 161
tensor([321.2789]) 204
tensor([330.1233]) 193
tensor([209.2000]) 112
tensor([293.2592]) 212
tensor([360.0347]) 158
tensor([292.0163]) 107
tensor([215.0670]) 95
tensor([285.9044]) 135
tensor([236.8862]) 164
tensor([228.1310]) 150
tensor([230.2863]) 189
tensor([257.1998]) 188
tensor([253.8197]) 138
tensor([218.9882]) 143
tensor([246.1270]) 157
tensor([258.6054]) 149
tensor([270.1025]) 185
tensor([243.4768]) 187
tensor([230.1258]) 192
tensor([286.0833]) 151
tensor([342.8449]) 156
tensor([328.5769]) 163
tensor([330.1681]) 118
tensor([334.0115]) 142
tensor([213.3468]) 161
tensor([305.6221]) 149
tensor([246.4794]) 160
tensor([235.7955]) 172
tensor([276.5143]) 164
tensor([241.05